# Імпорти

In [22]:
import pandas as pd
import numpy as np
import glob
import os
import csv

# Огляд файлів

In [23]:
start_data_path = 'data/accidentlvivmth2023.csv'

df = pd.read_csv(start_data_path, encoding='cp1251', sep = ';')

print(df.head(), '\n')
print(df.info(), '\n')
print(df.isnull().sum(), '\n')
print(df.nunique())

  accidentDate accidentTime accidentDay settlementType settlement  \
0   2023-01-09     20:10:00   понеділок          місто      Львів   
1   2023-01-31     11:03:00    вівторок          місто      Львів   
2   2023-01-13     23:26:00    п'ятниця          місто      Львів   
3   2023-01-20     21:12:00    п'ятниця          місто      Львів   
4   2023-01-15     18:52:00      неділя          місто      Львів   

       district streetType         streetName houseNumber  \
0  Франківський     вулиця        Кобилиці Л.          19   
1     Галицький     вулиця       Політехнічна           7   
2  Франківський     вулиця          Стрийська        280а   
3  Личаківський     вулиця  Глинянський Тракт           8   
4   Залізничний     вулиця     Кульпарківська         NaN   

                            locationComments  ...  \
0                                        NaN  ...   
1                                        NaN  ...   
2                                        NaN  ...   
3     

# Об'єднання файлів в один

In [24]:
csv_mix = os.path.join('data', '*.csv')

file_list = glob.glob(csv_mix)

print(f'{len(file_list)} files found ')
print(file_list)


8 files found 
['data\\accidentlviv2017.csv', 'data\\accidentlviv2018.csv', 'data\\accidentlviv2019.csv', 'data\\accidentlviv2020.csv', 'data\\accidentlvivmth2021.csv', 'data\\accidentlvivmth2022.csv', 'data\\accidentlvivmth2023.csv', 'data\\dtp2024public.csv']


In [25]:
list_of_files = []

for file in file_list:

    with open(file, 'r', encoding='cp1251') as f:

        header_line = f.readline()
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(header_line)
        delimiter = dialect.delimiter
        print(f'{delimiter} : {os.path.basename(file)}')

    df_temp = pd.read_csv(
        file,
        encoding='cp1251',
        sep = delimiter,
        low_memory=False
    )

    list_of_files.append(df_temp)

df_combined = pd.concat(list_of_files, axis=0, ignore_index=True)
print(df_combined.info(), '\n')

df_combined.to_csv('combined_accidents.csv', sep = ';', index = False, encoding = 'utf-8-sig')

print(f'Combined accidents data saved to combined_accidents.csv')

, : accidentlviv2017.csv
, : accidentlviv2018.csv
, : accidentlviv2019.csv
, : accidentlviv2020.csv
, : accidentlvivmth2021.csv
, : accidentlvivmth2022.csv
; : accidentlvivmth2023.csv
; : dtp2024public.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5908 entries, 0 to 5907
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accidentDate               5908 non-null   object 
 1   accidentTime               5908 non-null   object 
 2   accidentDay                5908 non-null   object 
 3   settlementType             5908 non-null   object 
 4   settlement                 5908 non-null   object 
 5   district                   5898 non-null   object 
 6   streetType                 5897 non-null   object 
 7   streetName                 5891 non-null   object 
 8   houseNumber                4461 non-null   object 
 9   locationComments           983 non-null    object 
 10  accidentTy

# Підготовка даних

#### Зменшуємо к-сть колонок InjuryStatus і трансформує у колонки вигляду Count_'status'

In [26]:
df = pd.read_csv('combined_accidents.csv', sep = ';', low_memory=False)

injury_cols = df.filter(like='injuryStatusParticipant').columns

all_unique_values = pd.Series(df[injury_cols].values.ravel()).unique()
print("Unique item in InjuryStatusParticipation")
print(all_unique_values)

Unique item in InjuryStatusParticipation
['легко травмований' 'без ушкоджень' nan 'тяжко травмований'
 'помер на місці дтп' 'помер у лікарні протягом 30 діб'
 'помер по дорозі в лікарню']


In [27]:
injury_mapping = {
    'помер на місці дтп': 'Загинув',
    'помер у лікарні протягом 30 діб': 'Загинув',
    'помер по дорозі в лікарню': 'Загинув',

    'легко травмований': 'Легко травмований',
    'тяжко травмований': 'Тяжко травмований',
    'без ушкоджень': 'Без ушкоджень'
}

injury_counts_df = df.apply(lambda row: row[injury_cols].value_counts(), axis = 1)

injury_counts_df = injury_counts_df.fillna(0).astype(int)

injury_counts_df = injury_counts_df.rename(columns = {col: f'Count_{col}' for col in injury_counts_df.columns if col is not np.nan})



In [28]:
injury_cols = df.filter(like='injuryStatusParticipant').columns


print(f"Знайдено {len(injury_cols)} колонок зі статусами травм.")

df[injury_cols] = df[injury_cols].replace(injury_mapping)

all_unique_values = pd.Series(df[injury_cols].values.ravel()).unique()
print("\n--- Унікальні статуси ПІСЛЯ об'єднання ---")
print(all_unique_values)

print("\nПочинаю підрахунок об'єднаних статусів по кожному ДТП...")

injury_counts_df = df.apply(lambda row: row[injury_cols].value_counts(), axis=1)

injury_counts_df = injury_counts_df.fillna(0).astype(int)

injury_counts_df = injury_counts_df.rename(columns={
    col: f'Count_{col}' for col in injury_counts_df.columns if col is not np.nan
})

if np.nan in injury_counts_df.columns:
    injury_counts_df = injury_counts_df.drop(columns=[np.nan])
if 'Count_nan' in injury_counts_df.columns:
    injury_counts_df = injury_counts_df.drop(columns=['Count_nan'])

print("Створено нові колонки з підрахунками:")
print(injury_counts_df.head())

df= pd.concat([df, injury_counts_df], axis=1)

df = df.drop(columns=injury_cols)

print("\n--- Фінальна інформація про DataFrame (після очистки) ---")
df.info()


Знайдено 20 колонок зі статусами травм.

--- Унікальні статуси ПІСЛЯ об'єднання ---
['Легко травмований' 'Без ушкоджень' nan 'Тяжко травмований' 'Загинув']

Починаю підрахунок об'єднаних статусів по кожному ДТП...
Створено нові колонки з підрахунками:
   Count_Без ушкоджень  Count_Загинув  Count_Легко травмований  \
0                    1              0                        2   
1                    1              0                        5   
2                    1              0                        1   
3                    1              0                        0   
4                    1              0                        1   

   Count_Тяжко травмований  
0                        0  
1                        0  
2                        0  
3                        1  
4                        0  

--- Фінальна інформація про DataFrame (після очистки) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5908 entries, 0 to 5907
Data columns (total 19 columns):
 #   Column

#### Додаємо колонку вигляду "Чи трапилося дтп на перехресті"

In [29]:
df['Is_Intersection'] = df['locationComments'].str.contains(
    'перехрестя',
    case=False,
    na=False
).astype(int)

print(f"\nВсього знайдено рядків з 'перехрестя': {df['Is_Intersection'].sum()}")


Всього знайдено рядків з 'перехрестя': 478


#### Трансформуємо дату й час

In [30]:
df['accidentDate'] = pd.to_datetime(df['accidentDate'], format='%Y-%m-%d')

df['Year'] = df['accidentDate'].dt.year
df['Month'] = df['accidentDate'].dt.month
df['DayOfWeek'] = df['accidentDate'].dt.dayofweek # 0=Пон, 1=Вів, ..., 6=Нед

df['Hour'] = pd.to_datetime(df['accidentTime'], format='mixed').dt.hour

### Видаляємо непотрібні колонки

In [31]:
cols_to_drop = ['houseNumber', 'locationComments', 'streetType','accidentDate', 'accidentTime']

df = df.drop(columns=cols_to_drop)
print(f"Видалено непотрібні колонки: {cols_to_drop}")

Видалено непотрібні колонки: ['houseNumber', 'locationComments', 'streetType', 'accidentDate', 'accidentTime']


#### Заливаємо" NaN у категорійних фічах

In [32]:
cols_to_fill = ['district', 'streetName', 'mainAccidentCause']

for col in cols_to_fill:
    df[col] = df[col].fillna('Невідомо')
print(f"Заповнено NaN у колонках: {cols_to_fill}")

Заповнено NaN у колонках: ['district', 'streetName', 'mainAccidentCause']


#### Видаляємо рядки, які не мають заповнених координат ('latitude', 'longitude')

In [33]:
rows_before = len(df)
df = df.dropna(subset=['latitude', 'longitude'])
rows_after = len(df)
print(f"Видалено {rows_before - rows_after} рядків без координат.")

Видалено 3 рядків без координат.


In [34]:
print(df.nunique())

accidentDay                   7
settlementType                3
settlement                   19
district                      8
streetName                  495
accidentType                 41
mainAccidentCause           264
totalParticipantsAmount      11
latitude                   5166
longitude                  5136
Count_Без ушкоджень           7
Count_Загинув                 3
Count_Легко травмований      10
Count_Тяжко травмований       6
Is_Intersection               2
Year                          8
Month                        12
DayOfWeek                     7
Hour                         24
dtype: int64


In [35]:
print(df['settlement'].value_counts())
print(df['settlementType'].value_counts())

settlement
Львів               5687
Винники               56
Брюховичі             36
Малехів               28
Рудне                 23
Дубляни               15
Рясне-Руське          13
Підбірці               9
Зашків                 7
Великі Грибовичі       6
Пасіки-Зубрицькі       5
Гряда                  4
Лисиничі               4
Сокільники             4
Малі Підліски          3
Збиранка               2
Солонка                1
Підрясне               1
Завадів                1
Name: count, dtype: int64
settlementType
місто     5760
село        86
селище      59
Name: count, dtype: int64


In [36]:
df['settlement_grouped'] = df['settlement'].apply(lambda x: 'Львів' if x == 'Львів' else 'Передмістя')

df = df.drop(columns=['settlement', 'settlementType'])

print(df['settlement_grouped'].value_counts())

settlement_grouped
Львів         5687
Передмістя     218
Name: count, dtype: int64


In [37]:
def clean_accident_type(val):
    val = str(val).lower()
    if 'пішоход' in val:
        return 'Наїзд на пішохода'
    elif 'велосипед' in val:
        return 'Наїзд на велосипедиста'
    elif 'зіткнення' in val:
        return 'Зіткнення автомобілів'
    elif 'перешкод' in val or 'огородження' in val or 'дерево' in val or 'знак' in val:
        return 'Наїзд на перешкоду'
    elif 'пасажир' in val:
        return 'Падіння пасажира'
    elif 'перекидання' in val:
        return 'Перекидання'
    else:
        return 'Інше'

def clean_accident_cause(val):
    val = str(val).lower()
    if 'маневрування' in val:
        return 'Порушення маневрування'
    elif 'швидкост' in val:
        return 'Перевищення швидкості'
    elif 'дистанц' in val:
        return 'Недотримання дистанції'
    elif 'перехрест' in val:
        return 'Порушення проїзду перехресть'
    elif 'пішохід' in val: # Тут і про переходи, і про пішоходів
        return 'Порушення проїзду переходів'
    elif 'нетверезому' in val:
        return 'П\'яне водіння'
    else:
        return 'Інше'

# Застосовуємо очистку
df['Simple_Type'] = df['accidentType'].apply(clean_accident_type)
df['Simple_Cause'] = df['mainAccidentCause'].apply(clean_accident_cause)

# Дивимось результат
print("--- ТИПИ ---")
print(df['Simple_Type'].value_counts())
print("\n--- ПРИЧИНИ ---")
print(df['Simple_Cause'].value_counts())

--- ТИПИ ---
Simple_Type
Наїзд на пішохода         2852
Зіткнення автомобілів     1956
Падіння пасажира           325
Наїзд на перешкоду         322
Наїзд на велосипедиста     256
Інше                       107
Перекидання                 87
Name: count, dtype: int64

--- ПРИЧИНИ ---
Simple_Cause
Порушення маневрування          2227
Інше                            1226
Перевищення швидкості           1073
Порушення проїзду переходів      697
Порушення проїзду перехресть     365
Недотримання дистанції           285
П'яне водіння                     32
Name: count, dtype: int64


In [38]:
print(df['accidentType'].value_counts())
print(df['mainAccidentCause'].value_counts())
print(df['district'].value_counts())

accidentType
наїзд на пішохода - пішохід рухався у попутному напрямку                                                       1290
зіткнення бокове                                                                                               1139
наїзд на пішохода - на нерегульованому пішохідному переході                                                     523
зіткнення лобове                                                                                                444
наїзд на пішохода - пішохід рухався у зустрічному напрямку                                                      381
падіння пасажира                                                                                                325
наїзд на інші види перешкод                                                                                     288
зіткнення з задньою частиною тз                                                                                 222
наїзд на пішохода - на регульованому пішохідному переході  

In [39]:
df.to_csv('accident_clear_data.csv', sep=';', index=False, encoding='utf-8-sig')
print(f"\nЧистий DataFrame збережено як accident_clear_data.csv ")


Чистий DataFrame збережено як accident_clear_data.csv 
